# Gıda Listesini AL

In [1]:
URL="https://turkomp.tarimorman.gov.tr/database"

In [8]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import re
from urllib.parse import urljoin

In [6]:
soup = bs(requests.get(URL).content, "html.parser")


In [17]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Harf listesi
harfler = [
    "A", "B", "C", "Ç", "D", "E", "F", "G", "H", "I", "İ", "J", "K", "L", "M", "N",
    "O", "P", "R", "S", "Ş", "T", "U", "Ü", "V", "Y", "Z"
]

base_url = "https://turkomp.tarimorman.gov.tr/database?type=foods&harf="
root_url = "https://turkomp.tarimorman.gov.tr/"

veriler = []

for harf in harfler:
    url = base_url + harf
    print(f"İşleniyor: {url}")
    
    try:
        response = requests.get(url)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, "html.parser")
        
        rows = soup.select('table#mydatalist tbody tr')
        
        for row in rows:
            cols = row.find_all('td')
            if len(cols) < 3:
                continue

            gida_a_tag = cols[0].find('a')
            gida = gida_a_tag.text.strip()
            gida_link = root_url + gida_a_tag['href'].replace("&amp;", "&")

            kod = cols[1].text.strip()

            grup_links = cols[2].find_all('a')
            gruplar = [a.text.strip(" »").strip() for a in grup_links]
            grup_linkler = [root_url + a['href'].replace("&amp;", "&") for a in grup_links]

            veriler.append({
                'Gıda': gida,
                'Kod': kod,
                'Gruplar': ', '.join(gruplar),
                'Grup Linkleri': ', '.join(grup_linkler),
                'Gıda Linki': gida_link
            })
        
        time.sleep(1)  # siteyi yormamak için bekleme

    except Exception as e:
        print(f"Hata oluştu: {e}")

# CSV'ye yaz
with open('turkomp_gida_listesi.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Gıda', 'Kod', 'Gruplar', 'Grup Linkleri', 'Gıda Linki']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for item in veriler:
        writer.writerow(item)

print("Tüm veriler başarıyla CSV dosyasına yazıldı: turkomp_gida_listesi.csv")


İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=A
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=B
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=C
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=Ç
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=D
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=E
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=F
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=G
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=H
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=I
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=İ
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=J
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods&harf=K
İşleniyor: https://turkomp.tarimorman.gov.tr/database?type=foods

# Gıda Detayıı Alma

In [18]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [19]:
df = pd.read_csv('turkomp_gida_listesi.csv', encoding='utf-8')
df.head()

,Gıda,Kod,Gruplar,Grup Linkleri,Gıda Linki
0,Acur,08.01.0001,Sebze ve sebze ürünleri,https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-acur-204
1,Afyon pastırması,12.02.0073,"Et ve et ürünleri, Geleneksel gıdalar",https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-afyon-p...
2,Ahududu,09.01.0001,Meyve ve meyve ürünleri,https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-ahududu...
3,"Ahududu, dondurulmuş",09.02.0001,Meyve ve meyve ürünleri,https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-ahududu...
4,Ak sirken,08.01.0096,Sebze ve sebze ürünleri,https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-ak-sirk...


In [21]:
gıda_link = df['Gıda Linki'][0]
gıda_link

'https://turkomp.tarimorman.gov.tr/food-acur-204'

In [ ]:
import pandas as pd
import json
import time
import requests
from bs4 import BeautifulSoup

# Gıda sayfasından bileşen verilerini çeken fonksiyon
def extract_from_url(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        table = soup.find('table', {'id': 'foodResultlist'})
        if not table:
            return None
        
        data = []
        for row in table.select('tbody tr'):
            cols = row.find_all('td')
            if len(cols) >= 5:
                bileşen = cols[0].get_text(strip=True)
                birim = cols[1].get_text(strip=True)
                ort = cols[2].get_text(strip=True).replace(',', '.')
                min_ = cols[3].get_text(strip=True).replace(',', '.')
                max_ = cols[4].get_text(strip=True).replace(',', '.')
                
                def to_float(val):
                    try:
                        return float(val)
                    except:
                        return val
                
                data.append({
                    'Bileşen': bileşen,
                    'Birim': birim,
                    'Ortalama': to_float(ort),
                    'Minimum': to_float(min_),
                    'Maksimum': to_float(max_)
                })
        return data
    except Exception as e:
        return {"hata": str(e)}

# Ana işlem
def process_all_gida(csv_path_in, csv_path_out):
    df = pd.read_csv(csv_path_in)
    
    # Eğer link sütunu adı farklıysa buradan kontrol et
    if "Gıda Linki" in df.columns:
        link_col = "Gıda Linki"
    elif "Linkler" in df.columns:
        link_col = "Linkler"
    else:
        raise Exception("Gıda sayfası linkini içeren bir sütun bulunamadı!")
    
    df["Data"] = None

    for idx, row in df.iterrows():
        gida_adi = row["Gıda"]
        url_list = row[link_col].split(',')  # Bazı hücrelerde birden fazla link olabilir
        url = [u.strip() for u in url_list if "food" in u]  # Yalnızca gıda sayfası olanı seç

        if not url:
            print(f"[{idx}] {gida_adi} için uygun URL bulunamadı.")
            continue

        print(f"[{idx}] {gida_adi} - {url[0]}")
        data = extract_from_url(url[0])
        df.at[idx, "Data"] = json.dumps(data, ensure_ascii=False)

        time.sleep(1)  # siteyi yormamak için

    # Yeni CSV yaz
    df.to_csv(csv_path_out, index=False, encoding="utf-8-sig")
    print(f"\nTamamlandı! Yeni CSV dosyası: {csv_path_out}")

# Kullanım
if __name__ == "__main__":
    input_csv = "turkomp_gida_listesi.csv"  # İlk oluşturduğun dosya
    output_csv = "turkomp_gida_besinli.csv"  # Çıkış dosyası
    process_all_gida(input_csv, output_csv)


[0] Acur - https://turkomp.tarimorman.gov.tr/food-acur-204
[1] Afyon pastırması - https://turkomp.tarimorman.gov.tr/food-afyon-pastirmasi-518
[2] Ahududu - https://turkomp.tarimorman.gov.tr/food-ahududu-381

Tamamlandı! Yeni CSV dosyası: turkomp_gida_besinli.csv


In [2]:
import pandas as pd
gıda = pd.read_csv("/home/serdar/Documents/turkomp-g-da-veritabani/gıda-listesi/turkomp_gida_besinli.csv", encoding="utf-8-sig")
gıda.head()

,Gıda,Kod,Gruplar,Grup Linkleri,Gıda Linki,Status,Attempts,Last_Error,Data
0,Acur,08.01.0001,Sebze ve sebze ürünleri,https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-acur-204,Success,1.0,NaN,"[{""Bileşen"": ""Enerji"", ""Birim"": ""kcal"", ""Ortal..."
1,Afyon pastırması,12.02.0073,"Et ve et ürünleri, Geleneksel gıdalar",https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-afyon-p...,Success,1.0,NaN,"[{""Bileşen"": ""Enerji"", ""Birim"": ""kcal"", ""Ortal..."
2,Ahududu,09.01.0001,Meyve ve meyve ürünleri,https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-ahududu...,Success,1.0,NaN,"[{""Bileşen"": ""Enerji"", ""Birim"": ""kcal"", ""Ortal..."
3,"Ahududu, dondurulmuş",09.02.0001,Meyve ve meyve ürünleri,https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-ahududu...,Success,1.0,NaN,"[{""Bileşen"": ""Enerji"", ""Birim"": ""kcal"", ""Ortal..."
4,Ak sirken,08.01.0096,Sebze ve sebze ürünleri,https://turkomp.tarimorman.gov.tr/?type=foods&...,https://turkomp.tarimorman.gov.tr/food-ak-sirk...,Success,1.0,NaN,"[{""Bileşen"": ""Enerji"", ""Birim"": ""kcal"", ""Ortal..."


In [3]:
import json
json.loads(gıda.Data[0])

[{'Bileşen': 'Enerji',
  'Birim': 'kcal',
  'Ortalama': 22.0,
  'Minimum': 12.0,
  'Maksimum': 31.0},
 {'Bileşen': 'Enerji',
  'Birim': 'kJ',
  'Ortalama': 92.0,
  'Minimum': 51.0,
  'Maksimum': 129.0},
 {'Bileşen': 'Su',
  'Birim': 'g',
  'Ortalama': 93.81,
  'Minimum': 91.48,
  'Maksimum': 95.61},
 {'Bileşen': 'Kül',
  'Birim': 'g',
  'Ortalama': 0.58,
  'Minimum': 0.42,
  'Maksimum': 0.78},
 {'Bileşen': 'Protein',
  'Birim': 'g',
  'Ortalama': 0.65,
  'Minimum': 0.42,
  'Maksimum': 1.19},
 {'Bileşen': 'Azot',
  'Birim': 'g',
  'Ortalama': 0.1,
  'Minimum': 0.07,
  'Maksimum': 0.19},
 {'Bileşen': 'Yağ, toplam',
  'Birim': 'g',
  'Ortalama': 0.47,
  'Minimum': 0.12,
  'Maksimum': 0.71},
 {'Bileşen': 'Karbonhidrat',
  'Birim': 'g',
  'Ortalama': 3.03,
  'Minimum': 0.69,
  'Maksimum': 4.51},
 {'Bileşen': 'Lif, toplam diyet',
  'Birim': 'g',
  'Ortalama': 1.45,
  'Minimum': 1.19,
  'Maksimum': 1.89},
 {'Bileşen': 'Lif, suda çözünür',
  'Birim': 'g',
  'Ortalama': 0.42,
  'Minimum': 0.35,